In [ ]:
from sklearn.linear_model import SGDRegressor  # Импортируем регрессор SGD
from sklearn.model_selection import GridSearchCV  # Импортируем класс для поиска по сетке
from sklearn.model_selection import train_test_split  # Импортируем функцию для разделения данных на обучающую и тестовую выборки
from sklearn.metrics import mean_squared_error  # Импортируем функцию для вычисления среднеквадратичной ошибки
import seaborn as sns  # Импортируем библиотеку Seaborn для визуализации данных
import pandas as pd  # Импортируем библиотеку Pandas для работы с данными в формате DataFrame
import numpy as np  # Импортируем библиотеку NumPy для работы с массивами

# Загрузите стандартный датасет об алмазах из библиотеки Seaborn:
df = sns.load_dataset('diamonds')  # Загружаем датасет о алмазах

# Удалите часть признаков:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)  # Удаляем ненужные признаки для упрощения модели

# Закодируйте категориальные признаки:
df = pd.get_dummies(df, drop_first=True)  # Преобразуем категориальные переменные в числовые с помощью one-hot кодирования, исключая первую категорию

# Логарифмируйте признаки:
df['carat'] = np.log(1 + df['carat'])  # Применяем логарифмическое преобразование к признаку 'carat'
df['price'] = np.log(1 + df['price'])  # Применяем логарифмическое преобразование к целевой переменной 'price'

# Определите целевую переменную и предикторы:
X_cols = [col for col in df.columns if col != 'price']  # Создаем список признаков, исключая целевую переменную 'price'
X = df[X_cols]  # Определяем матрицу предикторов
# Как альтернатива отделения признаков
# X = df.drop(columns="price")  # Альтернативный способ определения предикторов, удаляя столбец 'price'
y = df['price']  # Определяем целевую переменную

# Разделяем данные на обучающую и тестовую выборки:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)  
# Разделяем данные на обучающую (67%) и тестовую (33%) выборки с фиксированным состоянием генератора случайных чисел

# Определяем параметры для поиска по сетке:
parameters = {
    "loss": ["squared_error", "epsilon_insensitive"],  # Возможные функции потерь
    "penalty": ["elasticnet"],  # Тип регуляризации
    "alpha": np.logspace(-3, 3, 10),  # Значения параметра регуляризации alpha в логарифмическом масштабе
    "l1_ratio": np.linspace(0, 1, 10),  # Соотношение L1 и L2 в регуляризации
    "learning_rate": ["constant"],  # Фиксированная скорость обучения
    "eta0": np.logspace(-4, -1, 4)   # Значения начальной скорости обучения eta0 в логарифмическом масштабе
}

sgd = SGDRegressor(random_state=42)  # Инициализируем регрессор SGD с фиксированным состоянием генератора случайных чисел
sgd_cv = GridSearchCV(estimator=sgd, param_grid=parameters, n_jobs=-1)  
# Инициализируем поиск по сетке с использованием SGDRegressor и заданными параметрами; n_jobs=-1 позволяет использовать все доступные ядра процессора

sgd_cv.fit(X_train, y_train)  # Обучаем модель на обучающей выборке

print(sgd_cv.best_params_)  # Выводим лучшие параметры модели после поиска по сетке

sgd = SGDRegressor(**sgd_cv.best_params_, random_state=42)  
# Инициализируем новый регрессор SGD с лучшими параметрами из поиска по сетке

sgd.fit(X_train, y_train)  # Обучаем модель на обучающей выборке с лучшими параметрами
sgd.score(X_train, y_train)  # Оцениваем модель на обучающей выборке (вычисляем коэффициент детерминации R^2)

ls = sgd.predict(X_test)  # Предсказываем значения целевой переменной на тестовой выборке

round(mean_squared_error(y_test, ls), 3)  # Вычисляем и округляем среднеквадратичную ошибку между предсказанными и истинными значениями на тестовой выборке

{'alpha': np.float64(0.001), 'eta0': np.float64(0.001), 'l1_ratio': np.float64(0.0), 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


np.float64(0.044)